**Purpose of Notebook:** **Importing excel Tables from** '<u><mark>Data Tables .xlxs</mark></u>' **file using Pandas**

**kernal Python 3**

**Created by:** <span style="background-color:rgba(127, 127, 127, 0.1);">Shabba</span>

**Date:** <span style="background-color:rgba(127, 127, 127, 0.1);">23-05-202</span><span style="background-color:rgba(127, 127, 127, 0.1);">3</span>

1\. Provide a <mark>Database</mark> to be used and the <mark>Data Table/Config file</mark> Location and Properties (i.e., filename & Sheet name)

In [26]:
strConfigFileLocation = r"C:\Users\NkululekoTshabalala\DBK Advisory Dropbox\Nkululeko Tshabalala\Github Script Mshengu\config.ini" ## note, the 'r' needs to stay in front to indicate this is a location. 

fileName = r"\Data Tables.xlsx" ## The filename in the location as specified in your configuration file
sheetName = 'Weightings' ## The sheetname in the filename supplied above
database = 'Mshengu_Repository' ## the database where all these operations should take place
sqlSchemaName = "Refdata"

print("Complete")

Complete


2\. Install dependent packages on Azure Data Studio (if not done yet) \>\> Manage Packages \>\> Add New \>\> Pandas, xlrd, openpyxl, pyodbc, configparser

3\. Import libraries

In [27]:
import pandas as pd
import pyodbc
import configparser
print("omplete") 

omplete


4\. Cosmetics: Set the column width to display all the text to ensure that we are not cutting off any part of a cell

In [28]:
pd.set_option("display.max_colwidth", None)

print("Complete")

Complete


5\. Get the file location from the config file <mark>\[Get the <u>Data Tables</u> \]</mark>

In [29]:
strConfigFileLocation = r"C:\Users\NkululekoTshabalala\DBK Advisory Dropbox\Nkululeko Tshabalala\Github Script Mshengu\Data Tables Folder"
location=strConfigFileLocation+fileName
print(location)
print("Complete")

C:\Users\NkululekoTshabalala\DBK Advisory Dropbox\Nkululeko Tshabalala\Github Script Mshengu\Data Tables Folder\Data Tables.xlsx
Complete


6\. Import the Data Tables from file location

In [30]:
# using pandsas df to import the excel file, name your file after "df" (Create a similar cell like this for other tables & Add thier SheetName in the fisrt step)
dfDataTable = pd.read_excel(location, sheet_name= sheetName, dtype=str,skiprows=1)
print("Complete & File Imported successfully")

Complete & File Imported successfully


7\. Display the Imported data and <mark>Data Cleaning</mark>

In [31]:
dfDataTable.columns ## Show all the Columns imported

Index(['Weight_ID', 'AMKEY', 'SDG_1', 'SDG_2', 'SDG_3', 'SDG_4', 'SDG_5',
       'SDG_CHECK'],
      dtype='object')

In [32]:
dfDataTable.applymap(type) ## Show Data type per column

,Weight_ID,AMKEY,SDG_1,SDG_2,SDG_3,SDG_4,SDG_5,SDG_CHECK
0,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'float'>,<class 'str'>,<class 'str'>
1,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'float'>,<class 'float'>,<class 'str'>
2,<class 'str'>,<class 'str'>,<class 'float'>,<class 'float'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>
3,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'float'>,<class 'str'>
4,<class 'str'>,<class 'str'>,<class 'str'>,<class 'float'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>


In [33]:
dfDataTable ## Show data inside the Table

,Weight_ID,AMKEY,SDG_1,SDG_2,SDG_3,SDG_4,SDG_5,SDG_CHECK
0,1,1,0.6,0.1,0.15,NaN,0.15,1
1,2,2,0.15,0.6,0.25,NaN,NaN,1
2,3,3,NaN,NaN,0.15,0.75,0.1,1
3,4,4,0.6,0.15,0.1,0.15,NaN,1
4,5,5,0.1,NaN,0.6,0.15,0.15,1


In [34]:
dfDataTable = dfDataTable.where(pd.notnull(dfDataTable),None) ## convertion all the NaN to None
dfDataTable ## Check the changes from the Above Step

,Weight_ID,AMKEY,SDG_1,SDG_2,SDG_3,SDG_4,SDG_5,SDG_CHECK
0,1,1,0.6,0.1,0.15,None,0.15,1
1,2,2,0.15,0.6,0.25,None,None,1
2,3,3,None,None,0.15,0.75,0.1,1
3,4,4,0.6,0.15,0.1,0.15,None,1
4,5,5,0.1,None,0.6,0.15,0.15,1


**<u>8\. Adding Data to SQL Server</u>**

8.1 Connecting to the Database

In [35]:
connConfig = configparser.ConfigParser()
connConfig.read(strConfigFileLocation)
server = "13.246.56.66"
username = "tshaba"
password = "y4p8BOQkkUejyrDv"
port = "1435"
cnxnStringSqlAlchemy = "mssql+pyodbc://"+username+":"+password+"@"+server+"/"+database+ "?driver=ODBC+Driver+17+for+SQL+Server"
cnxnString = 'DRIVER={ODBC Driver 17 for SQL Server};SERVER='+server+','+port+';DATABASE='+database+';UID='+username+';PWD='+ password
cnxn = pyodbc.connect(cnxnString)

print("Connected to the Mshengu Repository Successfully")

Connected to the Mshengu Repository Successfully


8.2 Import Data Into SQL tables from DataFrame.

In [ ]:
df=dfDataTable
cursor = cnxn.cursor()
op= "TRUNCATE TABLE [Refdata].[SGD_Weigtings]" ## Clean the table by deeting everything inside the table
cursor.execute(op)

query = "SET IDENTITY_INSERT [Refdata].[SGD_Weigtings] ON; INSERT INTO [Refdata].[SGD_Weigtings] ([Weight_ID],[AMKEY],[SDG_1],[SDG_2],[SDG_3],[SDG_4],[SDG_5]) VALUES(?,?,?,?,?,?,?)"

for index, row in df.iterrows():

    ## Weightings Table
    data = df['Weight_ID'].at[index],df['AMKEY'].at[index],df['SDG_1'].at[index],df['SDG_2'].at[index],df['SDG_3'].at[index],df['SDG_4'].at[index],df['SDG_5'].at[index]
    cursor.execute(query,data)
cursor.commit()
cursor.close()

print("Data successfully uploaded to SQL Database")